In [4]:
import psycopg2
from psycopg2 import sql
def create_admin_and_database():
    try:
        connection = psycopg2.connect(
            host="localhost",
            database="postgres",
            user="postgres",
            password="ayush05"
        )
        connection.autocommit = True 
        cursor = connection.cursor()
        cursor.execute("CREATE USER admin WITH PASSWORD 'ayush111';")
        cursor.execute("CREATE DATABASE hospital_management;")
        cursor.execute("GRANT ALL PRIVILEGES ON DATABASE hospital_management TO admin;")
        cursor.execute("ALTER DATABASE hospital_management OWNER TO admin;")
        print("Admin user and database created successfully.")

    except psycopg2.Error as e:
        print(f"Error: {e}")
    
    finally:
        if connection:
            cursor.close()
            connection.close()

create_admin_and_database()


Admin user and database created successfully.


In [8]:
def create_tables():
    try:
        connection = psycopg2.connect(
            host="localhost",
            database="hospital_management",
            user="admin",
            password="ayush111"
        )
        cursor = connection.cursor()

        queries = [
            """
            CREATE TABLE Address (
                AddressID SERIAL PRIMARY KEY,
                Street VARCHAR(100),
                City VARCHAR(50),
                State VARCHAR(50),
                ZipCode VARCHAR(10)
            );
            """,
            """
            CREATE TABLE Department (
                DepartmentID SERIAL PRIMARY KEY,
                DepartmentName VARCHAR(100) NOT NULL UNIQUE
            );
            """,

            """
            CREATE TABLE Specialization (
                SpecializationID SERIAL PRIMARY KEY,
                SpecializationName VARCHAR(100) NOT NULL UNIQUE,
                DepartmentID INT REFERENCES Department(DepartmentID) ON DELETE SET NULL
            );
            """,

            """
            CREATE TABLE UserAccount (
                UserID SERIAL PRIMARY KEY,
                PhoneNumber VARCHAR(10) NOT NULL,
                PasswordHash TEXT NOT NULL,
                Role VARCHAR(20) NOT NULL CHECK (Role IN ('Patient', 'Doctor', 'Reception')),
                Status VARCHAR(10) DEFAULT 'Active' CHECK (Status IN ('Active', 'Deactivated')),
                CreatedAt TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
                UpdatedAt TIMESTAMP DEFAULT CURRENT_TIMESTAMP
            );
            """,

            """ 
            CREATE UNIQUE INDEX unique_phone_active 
            ON UserAccount (PhoneNumber) 
            WHERE Status = 'Active';
            """,

            """
            CREATE TABLE Insurance (
                InsuranceID SERIAL PRIMARY KEY,
                InsuranceProviderName VARCHAR(100),
                PolicyNumber VARCHAR(50) UNIQUE
            );
            """,

            """
            CREATE TABLE Patient (
                PatientID SERIAL PRIMARY KEY,
                UserID INT UNIQUE REFERENCES UserAccount(UserID) ON DELETE SET NULL,
                FirstName VARCHAR(50) NOT NULL,
                LastName VARCHAR(50) NOT NULL,
                DateOfBirth DATE NOT NULL,
                Gender VARCHAR(10) NOT NULL CHECK (Gender IN ('Male', 'Female', 'Other')),
                AddressID INT REFERENCES Address(AddressID) ON DELETE SET NULL,
                Email VARCHAR(100) NOT NULL,
                EmergencyContact VARCHAR(15) NOT NULL,
                InsuranceID INT REFERENCES Insurance(InsuranceID) ON DELETE SET NULL
            );
            """,

            """
            CREATE TABLE Doctor (
                DoctorID SERIAL PRIMARY KEY,
                UserID INT UNIQUE REFERENCES UserAccount(UserID) ON DELETE SET NULL,
                FirstName VARCHAR(50) NOT NULL,
                LastName VARCHAR(50) NOT NULL,
                SpecializationID INT NOT NULL REFERENCES Specialization(SpecializationID) ON DELETE RESTRICT,
                AddressID INT NOT NULL REFERENCES Address(AddressID) ON DELETE CASCADE,
                Email VARCHAR(100) NOT NULL 
            );
            """,

            """
            CREATE TABLE Appointment (
                AppointmentID SERIAL PRIMARY KEY,
                PatientID INT NOT NULL REFERENCES Patient(PatientID) ON DELETE CASCADE,
                DoctorID INT NOT NULL REFERENCES Doctor(DoctorID) ON DELETE CASCADE,
                AppointmentDate DATE NOT NULL,
                StartTime TIME NOT NULL,
                EndTime TIME NOT NULL,
                Status VARCHAR(20) DEFAULT 'Scheduled' 
                    CHECK (Status IN ('Scheduled', 'Completed', 'Cancelled')),
                CHECK (StartTime < EndTime)
            );
            """,

            """
            CREATE TABLE MedicalRecord (
                RecordID SERIAL PRIMARY KEY,
                PatientID INT REFERENCES Patient(PatientID) ON DELETE SET NULL,
                DoctorID INT REFERENCES Doctor(DoctorID) ON DELETE SET NULL,
                AppointmentID INT REFERENCES Appointment(AppointmentID) ON DELETE SET NULL,
                RecordDate DATE NOT NULL,
                Diagnosis TEXT,
                Prescription TEXT,
                TestResults TEXT
            );
            """,

            """
            CREATE TABLE Billing (
                BillingID SERIAL PRIMARY KEY,
                PatientID INT NOT NULL REFERENCES Patient(PatientID) ON DELETE CASCADE,
                AmountCharged DECIMAL(10, 2) NOT NULL,
                PaymentStatus VARCHAR(20) DEFAULT 'Pending' 
                    CHECK (PaymentStatus IN ('Pending', 'Paid', 'Insurance Claimed')),
                BillingDate DATE NOT NULL
            );
            """,

            """
            CREATE TABLE Supplier (
                SupplierID SERIAL PRIMARY KEY,
                SupplierName VARCHAR(100),
                ContactPerson VARCHAR(50),
                PhoneNumber VARCHAR(10),
                Email VARCHAR(100)
            );
            """,

            """
            CREATE TABLE InventoryItem (
                ItemID SERIAL PRIMARY KEY,
                ItemName VARCHAR(100),
                Category VARCHAR(50),
                QuantityInStock INT NOT NULL,
                ReorderLevel INT NOT NULL,
                SupplierID INT REFERENCES Supplier(SupplierID) ON DELETE SET NULL
            );
            """,

            """
            CREATE TABLE PharmacyPrescription (
                PrescriptionID SERIAL PRIMARY KEY,
                PatientID INT NOT NULL REFERENCES Patient(PatientID) ON DELETE CASCADE,
                DoctorID INT NOT NULL REFERENCES Doctor(DoctorID) ON DELETE CASCADE,
                AppointmentID INT REFERENCES Appointment(AppointmentID) ON DELETE SET NULL,
                MedicationName VARCHAR(100),
                DosageInstructions TEXT,
                DateIssued DATE NOT NULL
            );
            """,

            """
            CREATE TABLE LaboratoryTest (
                TestID SERIAL PRIMARY KEY,
                PatientID INT NOT NULL REFERENCES Patient(PatientID) ON DELETE CASCADE,
                TestType VARCHAR(100) NOT NULL,
                TestDate DATE NOT NULL,
                Results TEXT
            );
            """,

            """
            CREATE TABLE Staff (
                StaffID SERIAL PRIMARY KEY,
                FirstName VARCHAR(50) NOT NULL,
                LastName VARCHAR(50) NOT NULL,
                DepartmentID INT REFERENCES Department(DepartmentID) ON DELETE SET NULL,
                AddressID INT NOT NULL REFERENCES Address(AddressID) ON DELETE SET NULL,
                PhoneNumber VARCHAR(10),
                Email VARCHAR(100)
            );
            """
        ]

        for query in queries:
            cursor.execute(query)

        connection.commit()
        print("All tables created successfully.")

    except psycopg2.Error as e:
        print(f"Error: {e}")
        connection.rollback()

    finally:
        if connection:
            cursor.close()
            connection.close()

create_tables()


All tables created successfully.
